### Importing data

In [1]:
import numpy as np
import pandas as pd
import random
from numpy import array,identity,diagonal
from math import sqrt

In [2]:
movie_data = pd.io.parsers.read_csv('movies.dat',
    names=['movie_id', 'title', 'genre'],
    engine='python', delimiter='::')

In [3]:
movie_data

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
94,96,In the Bleak Midwinter (1995),Comedy
95,97,"Hate (Haine, La) (1995)",Drama
96,98,Shopping (1994),Action|Thriller
97,99,Heidi Fleiss: Hollywood Madam (1995),Documentary


In [4]:
data = pd.DataFrame(columns = ['user_id', 'movie_id', 'rating', 'time'])

In [5]:
data

,user_id,movie_id,rating,time


In [6]:
for i in range(200):
    userid = random.randint(1,15)
    movieid = random.randint(1,100)
    rating = random.randint(1,5)
    time = random.randint(30,150)
    row = [userid, movieid, rating, time]
    data.loc[len(data)] = row

In [7]:
data

,user_id,movie_id,rating,time
0,11,42,1,127
1,12,74,1,76
2,10,93,3,79
3,14,93,3,141
4,5,67,5,59
...,...,...,...,...
195,8,65,4,124
196,10,32,1,131
197,8,68,5,93
198,8,60,2,31


In [32]:
data['rating'].unique()

array([1, 3, 5, 2, 4], dtype=int64)

### Data preprocessing

In [8]:
data = data.astype('int64')

In [9]:
# Create the ratings matrix of shape (movies × users) with rows as movies and columns as users
ratings = np.ndarray(
                shape=(np.max(data.movie_id.values), np.max(data.user_id.values)),
                dtype=np.uint8)
ratings[data.movie_id.values-1, data.user_id.values-1] = data.rating.values

In [11]:
ratings.shape

(100, 15)

In [12]:
# Normalise matrix 
normalised_ratings = ratings - np.asarray([(np.mean(ratings, 1))]).T

In [13]:
normalised_ratings

array([[110.46666667,  60.46666667, -62.53333333, ..., -64.53333333,
        -64.53333333, -65.53333333],
       [ -0.33333333,   1.66666667,  -0.33333333, ...,  -0.33333333,
         -0.33333333,  -0.33333333],
       [ -0.66666667,  -0.66666667,   3.33333333, ...,  -0.66666667,
         -0.66666667,   0.33333333],
       ...,
       [ -7.86666667,  -9.86666667, -12.86666667, ..., -12.86666667,
        -12.86666667, -12.86666667],
       [-25.4       , -25.4       , 158.6       , ..., -23.4       ,
        -25.4       , -25.4       ],
       [-25.26666667, -25.26666667, -25.26666667, ..., -25.26666667,
        -24.26666667, -20.26666667]])

In [14]:
normalised_ratings.shape

(100, 15)

### Computing eigen pairs using Jacobi method

In [15]:
covariance = np.cov(normalised_ratings)

In [16]:
covariance

array([[ 5.40169524e+03,  2.50238095e+01, -1.41666667e+01, ...,
         1.04257619e+03, -4.93871429e+02,  7.19133333e+02],
       [ 2.50238095e+01,  8.09523810e-01, -2.38095238e-01, ...,
         3.50476190e+01, -8.00000000e+00, -9.02380952e+00],
       [-1.41666667e+01, -2.38095238e-01,  2.52380952e+00, ...,
        -9.19047619e+00,  1.00500000e+02, -1.76904762e+01],
       ...,
       [ 1.04257619e+03,  3.50476190e+01, -9.19047619e+00, ...,
         2.21740952e+03, -2.84800000e+02, -3.48319048e+02],
       [-4.93871429e+02, -8.00000000e+00,  1.00500000e+02, ...,
        -2.84800000e+02,  4.17554286e+03, -6.21185714e+02],
       [ 7.19133333e+02, -9.02380952e+00, -1.76904762e+01, ...,
        -3.48319048e+02, -6.21185714e+02,  4.28678095e+03]])

In [17]:
covariance.shape

(100, 100)

In [18]:
# Actual eigen pairs
eigen_values, eigen_vectors = np.linalg.eig(covariance)

In [19]:
eigen_values[0:10]

array([20624.05347338+0.j, 10438.08483209+0.j, 19008.77204104+0.j,
       18164.66441671+0.j, 15775.08181561+0.j, 16810.41369003+0.j,
       16524.23165428+0.j,  2682.61676453+0.j,  1818.30624654+0.j,
         427.57373644+0.j])

In [27]:
# Eigen pairs by Jacobi method
def Jacobi(a, limit):
    def maxElement(a):
        n = len(a)
        max_ = 0.0
        for i in range(n-1):
            for j in range(i+1, n):
                if abs(a[i,j]) >= max_:
                    max_ = abs(a[i,j])
                    k = i
                    l = j
        return max_, k, l
    
    def rotate(A, p, k, l):
        n = len(a)
        diff = a[l,l] - a[k,k]
        if abs(a[k,l]) < abs(diff)*1.0e-36: 
            t = a[k,l]/diff
        else:
            phi = diff/(2.0*a[k,l])
            t = 1.0/(abs(phi) + sqrt(phi**2 + 1.0))
            if phi < 0.0: 
                t = -t
        c = 1.0/sqrt(t**2 + 1.0)
        s = t*c
        tau = s/(1.0 + c)
        temp = a[k,l]
        a[k,l] = 0.0
        a[k,k] = a[k,k] - t*temp
        a[l,l] = a[l,l] + t*temp
        for i in range(k): 
            temp = a[i,k]
            a[i,k] = temp - s*(a[i,l] + tau*temp)
            a[i,l] = a[i,l] + s*(temp - tau*a[i,l])
        for i in range(k+1, l): 
            temp = a[k,i]
            a[k,i] = temp - s*(a[i,l] + tau*a[k,i])
            a[i,l] = a[i,l] + s*(temp - tau*a[i,l])
        for i in range(l+1, n): 
            temp = a[k,i]
            a[k,i] = temp - s*(a[l,i] + tau*temp)
            a[l,i] = a[l,i] + s*(temp - tau*a[l,i])
        for i in range(n): 
            temp = p[i,k]
            p[i,k] = temp - s*(p[i,l] + tau*p[i,k])
            p[i,l] = p[i,l] + s*(temp - tau*p[i,l])
    
    n = len(a)
    max_rotations = 5*(n**2) 
    p = identity(n)*1.0 
    for i in range(max_rotations): 
        max_, k, l = maxElement(a)
        if max_ < limit: 
            return diagonal(a), p 
        rotate(a, p, k, l)
    print('Jacobi method did not converge')

jacobi_eigen_values, jacobi_eigen_vectors = Jacobi(covariance, limit=1.0e-9)

In [28]:
jacobi_eigen_values = list(jacobi_eigen_values)
jacobi_eigen_values.sort(reverse=True)
jacobi_eigen_values[0:10]

[20624.05347338157,
 19008.772041041557,
 18164.6644167113,
 16810.41369003035,
 16524.231654279254,
 15775.081815610298,
 10438.084832091447,
 2682.616764527335,
 1818.3062465366186,
 1258.1510589272802]

### Recommending top 5 movies

In [29]:
def top_cosine_similarity(topk_eigenvecs, movie_id, top_n):
    index = movie_id - 1 # Movie id starts from 1
    movie_row = topk_eigenvecs[index, :]
    magnitude = np.sqrt(np.einsum('ij, ij -> i', topk_eigenvecs, topk_eigenvecs))
    similarity = np.dot(movie_row, topk_eigenvecs.T) / (magnitude[index] * magnitude)
    sort_indexes = np.argsort(-similarity)
    return sort_indexes[:top_n]

# Helper function to print top N similar movies
def print_similar_movies(movie_data, movie_id, top_indexes):
    print('Recommendations for {0}: \n'.format(
    movie_data[movie_data.movie_id == movie_id].title.values[0]))
    for id in top_indexes + 1:
        print(movie_data[movie_data.movie_id == id].title.values[0])

In [30]:
k = 10
movie_id = 25
top_n = 5

topk_eigenvecs = jacobi_eigen_vectors[:, :k]
top_indexes = top_cosine_similarity(topk_eigenvecs, movie_id, top_n)
print_similar_movies(movie_data, movie_id, top_indexes)

Recommendations for Leaving Las Vegas (1995): 

Toy Story (1995)
MisÃ©rables, Les (1995)
Kicking and Screaming (1995)
Fair Game (1995)
From Dusk Till Dawn (1996)


<ipython-input-29-d6b783340310>:5: RuntimeWarning: invalid value encountered in true_divide
  similarity = np.dot(movie_row, topk_eigenvecs.T) / (magnitude[index] * magnitude)
